In [ ]:
import pandas as pd

In [ ]:
# Load dataset
url = "assets/declutter-gold_DevelopmentSet.csv"

In [ ]:
#define column lables
names = ['id', 'type', 'path_to_file', 'begin_line', 'link_to_comment', 'comment', 'non-information']

In [ ]:
#na_filter = false, because pandas read empty string as float (nan)!
#skiprows = 1, because the first row contains only the labels
dataframe = pd.read_csv(url, names=names, skiprows = 1, na_filter = False)

In [ ]:
def get_java_file_name(v):
	x = v.split('/')
	y = x[-1].split('#')
	javaFileName = y[0]
	return javaFileName

def get_comment_line(v):
	x = v.split('/')
	y = x[-1].split('#')
	line = y[1][1:]
	return int(line)

def get_code_from_javadoc_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        if not (comment.startswith('/**') and comment.endswith('*/')):
            while comment != '*/':
                comment = lines[commentStartLine + i].strip()
                i += 1
        firstCodeLine = lines[commentStartLine + i].strip()
    return firstCodeLine

def get_code_from_line_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        isComment = comment.startswith('//')
        if not isComment:
            code = comment.split('//')
            firstCodeLine = code[0].strip()
        else:
            while isComment:
                if 0 <= commentStartLine + i < len(lines):
                    comment = lines[commentStartLine + i].strip()
                    isComment = comment.startswith('//')
                    if isComment:
                        i += 1
                else:
                    isComment = False
            if 0 <= commentStartLine + i < len(lines):
                firstCodeLine = lines[commentStartLine + i].strip()
            else:
                firstCodeLine = ''
    return firstCodeLine 
    
def get_code_from_block_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        startsWithComment = comment.startswith('/*')
        if not startsWithComment:
            code = comment.split('/*')
            firstCodeLine = code[0].strip()
        else:
            while comment != '*/':
                comment = lines[commentStartLine + i].strip()
                i += 1
            firstCodeLine = lines[commentStartLine + i].strip()
    return firstCodeLine

## extract One line code after comment
## Outfile: Paht to output CSV
## Javadir: dir of Java files
## csvPath: csv Database from project
def extractCode(Outfile,Javadir,csvPath):
    dataframe = pd.read_csv(csvPath, na_filter = False)
    names = ['id', 'type', 'comment', 'code','non-information']
    df = pd.DataFrame()
    for i,data in dataframe.iterrows():
#         print(i)
        link = data["link_to_comment"]
        file_type = data["type"]
        javaFileName = get_java_file_name(link)
        line = get_comment_line(link)

        f = open(Javadir+"/" + javaFileName, "r")
        lines = f.readlines()
        line = line - 1
        code = 'undefined type'
        if file_type == 'Javadoc':
            code = get_code_from_javadoc_comment(lines, line)
        elif file_type == 'Line':
            code = get_code_from_line_comment(lines, line)
        elif file_type == 'Block':
            code = get_code_from_block_comment(lines, line)

    #     print("Code:",code)
    #     print()
        df = df.append(pd.DataFrame([[data["ID"],data["type"],data["comment"],code,data["non-information"]]], columns=names))
        f.close()
#     print(df)
    df.to_csv(Outfile, index=False)

In [ ]:
data = dataframe[['type', 'link_to_comment']]
idxx = 0
for row in data.itertuples():
    print(idxx)
    type = row.type
    link = row.link_to_comment
    
    javaFileName = get_java_file_name(link)
    line = get_comment_line(link)
    
    print('type:', type)
    print('Java file:', javaFileName)
    print('Comment start line:', line)
	
    f = open("data/" + javaFileName, "r")
    lines = f.readlines()
    line = line - 1
    code = 'undefined type'
    if type == 'Javadoc':
        code = get_code_from_javadoc_comment(lines, line)
    elif type == 'Line':
        code = get_code_from_line_comment(lines, line)
    elif type == 'Block':
        code = get_code_from_block_comment(lines, line)
    
    print(code)
    print()
    idxx += 1

In [ ]:
Outfile ="csv/oneLineCode.csv"
Javadir ="data/"
csvPath = "assets/declutter-gold_DevelopmentSet.csv"
extractCode(Outfile,Javadir,csvPath)

In [ ]:
csvPath = "csv/oneLineCode.csv"
dataframe = pd.read_csv(csvPath, na_filter = False)
keep_col = ['comment','code','non-information']
new_f = dataframe[keep_col]
new_f.to_csv("csv/newFile.csv", index=False)
